In [1]:
from modelscope.msdatasets import MsDataset
from modelscope.utils.constant import DownloadMode

In [2]:
ms_train_dataset = MsDataset.load(
    'cats_and_dogs', namespace='tany0699',
    subset_name='default', split='train'
)
print(next(iter(ms_train_dataset)))

2024-08-11 16:14:27,418 - modelscope - WARNING - Reusing dataset dataset_builder (/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files)
2024-08-11 16:14:27,419 - modelscope - INFO - Generating dataset dataset_builder (/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files)
2024-08-11 16:14:27,419 - modelscope - INFO - Reusing cached meta-data file: /root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files/f393e0258171c4a8b950cbb19d29e9b7


{'image:FILE': '/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files/extracted/bc5712030f447cf9f432ea41ef6ea0ae14bb12e6fa549d38d259a7bc130d65ba/train/cat/Sphynx_159_jpg.rf.022528b23ac690c34ad5d109c1782079.jpg', 'category': 0}


In [3]:
ms_val_dataset = MsDataset.load(
    'cats_and_dogs', namespace='tany0699',
    subset_name='default', split='validation'
)
print(next(iter(ms_val_dataset)))

2024-08-11 16:14:34,113 - modelscope - WARNING - Reusing dataset dataset_builder (/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files)
2024-08-11 16:14:34,114 - modelscope - INFO - Generating dataset dataset_builder (/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files)
2024-08-11 16:14:34,114 - modelscope - INFO - Loading meta-data file ...
  1%|▏         | 71/4974 [00:00<00:00, 102794.47it/s]


100% {'image:FILE': '/root/.cache/modelscope/hub/datasets/tany0699/cats_and_dogs/master/data_files/extracted/69e4bbc7b1305e69fd4abed945da737a667815c44cc20a48d6ce6f9d6e0c34b3/val/cat/Persian_13_jpg.rf.dd6ccb81649242fcc8a22fd4ddbfbab7.jpg', 'category': 0}


In [1]:
import swanlab

In [2]:
swanlab.login("DDSBZOY03klTgFwA6u0X9")

In [3]:
num_epochs = 20
lr = 1e-4
batch_size = 8
num_classes = 2
device = 'cuda'
swanlab.init(
    experiment_name='ResNet50', 
    description='Train ResNet50 for Cats and Dogs classification.',
    config = {
        "model":"resnet50",
        "optim":"Adam",
        "lr":lr,
        "batch_size":batch_size,
        "num_epochs":num_epochs,
        "num_classes":num_classes,
        "device":device
    }
)

swanlab: \ Getting project...

swanlab: Tracking run with swanlab version 0.3.16                                  
swanlab: Run data will be saved locally in /home/code/ChatGLM3_llma3/Chapter3/swanlog/run-20240811_162330-b6f12368
swanlab: 👋 Hi Simon, welcome to swanlab!
swanlab: Syncing run ResNet50_Aug11_16-23-30 to the cloud
swanlab: 🌟 Run `swanlab watch -l /home/code/ChatGLM3_llma3/Chapter3/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@Simon/Chapter3
swanlab: 🚀 View run at https://swanlab.cn/@Simon/Chapter3/runs/23vnl25gg5dfyrvfdve74


In [5]:
import get_data
import torch

In [6]:
train_dataset = get_data.DatasetLoader(get_data.ms_train_dataset)
train_loader = (torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True))

In [7]:
import torch
import torchvision
from torchvision.models import ResNet50_Weights

In [8]:
model = torchvision.models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, num_classes)
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.5MB/s]


In [14]:
for images, labels in train_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    # print('Epoch [{}/{}], Iteration [{}/{}], Loss: {:.4f}'.format(
    #     iter + 1, num_epochs, iter + 1, len(train_loader), loss.item()
    # ))
    swanlab.log({"train_loss": loss.item()})

KeyError: 1